# Setup

In [1]:
import sys
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
path_to_root = '/content/drive/My Drive/Colab Notebooks/BatuEl_Dissertation'
sys.path.append(path_to_root)
print("Drive mounted.")

data_path = path_to_root + '/data'

Mounted at /content/drive
Drive mounted.


In [2]:
import torch
import tqdm
from reprshift.learning.algorithms import ERM, GroupDRO, Focal
from reprshift.models.hparams import hparams_f
from reprshift.dataset.datasets import MultiNLI, CivilComments
from reprshift.dataset.dataloaders import InfiniteDataLoader, FastDataLoader

# Data

In [9]:
hparams = hparams_f('GroupDRO')
hparams

{'batch_size': 32,
 'last_layer_dropout': 0.5,
 'optimizer': 'adamw',
 'weight_decay': 0.0001,
 'lr': 1e-05,
 'group_balanced': False,
 'num_training_steps': 30001,
 'num_warmup_steps': 0,
 'groupdro_eta': 0.01}

In [10]:
device = "cuda"
train_weights = None
batch_size = hparams['batch_size']

In [11]:
DATASET = 'MultiNLI'  # 'CivilComments' , 'MultiNLI'

if DATASET == 'MultiNLI':
    NUM_CLASSES = 3
    NUM_ATTRIBUTES = 2
    train_dataset = MultiNLI(data_path, 'tr', hparams)
    models_path = path_to_root + '/models/models_mnli'
    print(DATASET)
elif DATASET  == 'CivilComments':
    NUM_CLASSES = 2
    NUM_ATTRIBUTES = 8
    train_dataset = CivilComments(data_path, 'tr', hparams, granularity="fine")
    models_path = path_to_root + '/models/models_civilcomments'
    print(DATASET)
else:
    print('Dataset Not Implemented')

MultiNLI


In [12]:
### Dataloader ###
train_loader = InfiniteDataLoader(  dataset=train_dataset,
                                    weights=train_weights,
                                    batch_size=batch_size,
                                    num_workers=1)
steps_per_epoch = len(train_dataset) / batch_size

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


# Model

In [13]:
algorithm_name = 'GroupDRO'
random_seeds = [1,2] #[0,1,2]
init_state_dict_path = lambda random_seed : models_path + f'/00_randominit/seed{random_seed}/sd_epoch0.pth'
state_dict_PATH = models_path + '/03_groupdro/'

# Training

In [14]:
start_step = 1
n_steps = 30001 #hparams['num_training_steps']
checkpoint_freq = 1000
train_losses = {}

for seed in random_seeds:
    print('Training Seed:' , seed)
    algorithm = GroupDRO(num_classes=NUM_CLASSES, num_attributes=NUM_ATTRIBUTES, hparams=hparams)
    algorithm.to(device)
    init_state_dict = torch.load(init_state_dict_path(seed))
    init_state_dict['q'] = algorithm.state_dict()['q']
    algorithm.load_state_dict(init_state_dict)
    train_losses[seed] = []

    train_loader = InfiniteDataLoader(  dataset=train_dataset,
                                        weights=train_weights,
                                        batch_size=batch_size,
                                        num_workers=1)
    train_minibatches_iterator = iter(train_loader)

    for step in tqdm.tqdm(range(start_step, n_steps)):
        ### Training Step ###
        i, x, y, a = next(train_minibatches_iterator)
        minibatch_device = (i, x.to(device), y.to(device), a.to(device))
        algorithm.train()
        step_vals = algorithm.update(minibatch_device, step)
        train_losses[seed].append(step_vals['loss'])

        ### Evaluation ###
        if (step % checkpoint_freq == 0) or (step == n_steps - 1):
            epoch = int(step / checkpoint_freq)
            algorithm_state_dict = algorithm.state_dict()
            algorithm_state_dict_PATH = state_dict_PATH + f'seed{seed}/sd_epoch{epoch}.pth'
            torch.save(algorithm_state_dict, algorithm_state_dict_PATH)

    loss_PATH = state_dict_PATH + f'losses/Loss_{algorithm_name}_{seed}.pth'
    torch.save(train_losses, loss_PATH)

Training Seed: 1


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 30000/30000 [1:24:19<00:00,  5.93it/s]


Training Seed: 2


100%|██████████| 30000/30000 [1:24:23<00:00,  5.93it/s]
